## Sample Client Notebook

This Notebook demonstrates how to interact with the running service using http requests. The sample app provides a very primitive body segmentation algorithm which this Notebook will run.

### Setup

Before running this Notebook, follow these steps from your command line to get the sample service up and running:

1. Make sure Python libraries required are up-to-date:

```
pip install -r <PLATIPY_PATH>/requirements.txt
```

2. Change into the directory containing the sample app

```
cd <PLATIPY_PATH>/framework/sample
```

3. Initialize the database for the framework

```
python -m platipy.backend.manage initdb 
```

4. Add an API key for this client

```
python -m platipy.backend.manage key -a sample_client
```

Copy and paste the key output from the last command and copy into the api_key variable in the next cell!

5. Run the service

```
python sample.py
```

In [3]:
# Imports required for this notebook
import requests
import time
import json
import os
import sys
import pydicom
from pprint import pprint

sys.path.append("../../../") # Path containing PlatiPy library
from platipy.dicom.communication import DicomConnector

# API Key generated for this client (see above description)
api_key = 'fbf59863-6d63-4e1d-9c19-b2b167f08f46'

# URL at which the service is running
base_url = 'http://localhost:8000'

# The name of the algorithm running in the service
algorithm_name = 'Primitive Body Segmentation'

# These are the API endpoints which the client will use to communicate with the service
api_dicom_location = '{0}/api/dicomlocation'.format(base_url)
api_dataset = '{0}/api/dataset'.format(base_url)
api_dataset_ready = '{0}/api/dataset/ready'.format(base_url)
api_data_object = '{0}/api/dataobject'.format(base_url)
api_trigger = '{0}/api/trigger'.format(base_url)
api_algorithm = '{0}/api/algorithm'.format(base_url)

#### First, let's test that the service is up and running and that we can authenticate

We'll fetch the list of available algorithms from the service to achieve this

In [4]:
# Get the algorithm and the default settings
algorithm = None
r = requests.get(api_algorithm, headers={'API_KEY': api_key})
if r.status_code == 200:
    for a in r.json():
        pprint(a)
        if a['name'] == algorithm_name:
            algorithm = a
    print("")
    print("Look's Good!")
else:
    print("Oops, something went wrong. Ensure the service is running at the base_url configured and that the API Key has been generated and set in api_key.")

{'default_settings': {'lowerThreshold': -5000,
                      'outputContourName': 'primitive_body_contour',
                      'seed': [0, 0, 0],
                      'upperThreshold': -800,
                      'vectorRadius': [1, 1, 1]},
 'name': 'Primitive Body Segmentation'}

Look's Good!


### Create a Dataset

Next, we create a Dataset on the server

In [38]:
# Create a new Dataset
dataset = None
r = requests.post(api_dataset, headers={'API_KEY': api_key}, data={})
if r.status_code >= 200:
        dataset = r.json()
        
pprint(dataset)

{'from_dicom_location': None,
 'from_dicom_location_id': None,
 'id': 10,
 'input_data_objects': [],
 'output_data_objects': [],
 'owner_key': 'fbf59863-6d63-4e1d-9c19-b2b167f08f46',
 'timeout': '2022-05-31T08:14:55.509',
 'timestamp': '2022-05-30T08:14:55.517',
 'to_dicom_location': None,
 'to_dicom_location_id': None}


### Add Data Objects to the Dataset

Now that we have the Dataset, we want to add some data objects to it. In the case of this segmentation algorithm, all we need to add is one Dicom object (CT image series).

In [17]:
# Add a Dicom file to the dataset
path_to_ct = '/platipy/data/dicom/LCTSC-Test-S1-101/1.3.6.1.4.1.14519.5.2.1.7014.4598.106943890850011666503487579262'

# Get the Series UID of this Data Object
series_instance_UID = None
for f in os.listdir(path_to_ct):
    
    try:
        d = pydicom.read_file(os.path.join(path_to_ct, f))
        series_instance_UID = d.SeriesInstanceUID
    except:
        pass

data = {'dataset': dataset['id'],
        'type': 'DICOM',
        'dicom_retrieve': 'SEND',
        'seriesUID': series_instance_UID}
data_object = None
r = requests.post(api_data_object, headers={'API_KEY': api_key}, data=data)
if r.status_code >= 200:
        data_object = r.json()
        
pprint(data_object)

{'children': [],
 'dataset_id': 4,
 'id': 4,
 'is_fetched': False,
 'is_input': True,
 'is_sent': False,
 'meta_data': None,
 'parent': None,
 'parent_id': None,
 'path': None,
 'series_instance_uid': '1.3.6.1.4.1.14519.5.2.1.7014.4598.106943890850011666503487579262',
 'status': None,
 'timestamp': '2022-05-30T03:04:33.125',
 'type': 'DICOM'}


In [18]:
r.json()

{'children': [],
 'dataset_id': 4,
 'id': 4,
 'is_fetched': False,
 'is_input': True,
 'is_sent': False,
 'meta_data': None,
 'parent': None,
 'parent_id': None,
 'path': None,
 'series_instance_uid': '1.3.6.1.4.1.14519.5.2.1.7014.4598.106943890850011666503487579262',
 'status': None,
 'timestamp': '2022-05-30T03:04:33.125',
 'type': 'DICOM'}

### Send the Dicom Data to the Server

Now that the Data Object for the image series has been created on the server, we can send it the Dicom Data itself.

Since we set the Data Object's dicom_retrieve property to SEND, the server expects us to SEND the data object to it. If we set MOVE or GET, the Server will attempt to retrieve the Dicom object from the Dicom location configured.

In [19]:
# Setup and verify the Dicom Endpoint
dicom_connector = DicomConnector(host='127.0.0.1', port=7777)
dicom_connector.verify()

# Send the image series to the Dicom Location
img_series = [os.path.join(path_to_ct, f) for f in os.listdir(path_to_ct)]
dicom_connector.send_dcm(img_series)

2022-05-30 03:04:38.367 | DEBUG    | platipy.dicom.communication.connector:__init__:48 - DicomConnector with host: 127.0.0.1 port: 7777 AETitle: 
2022-05-30 03:04:39.341 | DEBUG    | platipy.dicom.communication.connector:send_dcm:334 - Sending file: ['/platipy/data/dicom/LCTSC-Test-S1-101/1.3.6.1.4.1.14519.5.2.1.7014.4598.106943890850011666503487579262/1-272.dcm', '/platipy/data/dicom/LCTSC-Test-S1-101/1.3.6.1.4.1.14519.5.2.1.7014.4598.106943890850011666503487579262/10-281.dcm', '/platipy/data/dicom/LCTSC-Test-S1-101/1.3.6.1.4.1.14519.5.2.1.7014.4598.106943890850011666503487579262/100-071.dcm', '/platipy/data/dicom/LCTSC-Test-S1-101/1.3.6.1.4.1.14519.5.2.1.7014.4598.106943890850011666503487579262/101-072.dcm', '/platipy/data/dicom/LCTSC-Test-S1-101/1.3.6.1.4.1.14519.5.2.1.7014.4598.106943890850011666503487579262/102-073.dcm', '/platipy/data/dicom/LCTSC-Test-S1-101/1.3.6.1.4.1.14519.5.2.1.7014.4598.106943890850011666503487579262/103-074.dcm', '/platipy/data/dicom/LCTSC-Test-S1-101/1.3.6

(0000, 0900) Status                              US: 0

### If we want to send the data objects as Nifti instead of Dicom...

The following cell demonstrates how to send the data as a Nifti object, bypassing the need for Dicom communication. Note the following is a sample and should only be run if the data 'type' above is set to 'FILE' .

In [27]:
import SimpleITK as sitk
load_path = sitk.ImageSeriesReader().GetGDCMSeriesFileNames(path_to_ct)
img = sitk.ReadImage(load_path)

path_to_nii = f'testcase.nii.gz'

sitk.WriteImage(img, path_to_nii)

data_object = None

# Get the Series UID of this Data Object
with open(path_to_nii,'rb') as file:

    data = {'dataset': dataset['id'],
            'type': 'FILE',
            'file_name': 'case_test.nii.gz'}
    
    r = requests.post(api_data_object, headers={'API_KEY': api_key}, data=data, files={'file_data':file})
    if r.status_code >= 200:
            data_object = r.json()
        
pprint(data_object)

{'children': [],
 'dataset_id': 5,
 'id': 6,
 'is_fetched': True,
 'is_input': True,
 'is_sent': False,
 'meta_data': None,
 'parent': None,
 'parent_id': None,
 'path': '/tmp/tmp15agy_lh/case_test.nii.gz',
 'series_instance_uid': None,
 'status': None,
 'timestamp': '2022-05-30T03:08:38.155',
 'type': 'FILE'}


### Refresh Data Object

Now we can refresh our Data Object from the Server, to see if it has been fetched yet or not. The is_fetched property tells us if it has been fetched or not.

In [28]:
r = requests.get('{0}/{1}'.format(api_data_object, data_object['id']), headers={'API_KEY': api_key})
if r.status_code == 200:
    data_object = r.json()
    
if data_object['is_fetched']:
    print('The server has the Dicom data and is ready!')
else:
    print('The server is still receiving the Dicom data or something has gone wrong.')

The server has the Dicom data and is ready!


### Check if Dataset ready

Our dataset only has one object, but when you have multiple objects it can be useful to determine if the dataset is ready to run the algorithm on. So, determine if all objects within the dataset have been fetched.

In [29]:
r = requests.get('{0}/{1}'.format(api_dataset_ready, dataset['id']), headers={'API_KEY': api_key})
if r.status_code == 200:
    if r.json()['ready']:
        print("The Dataset is ready, let's run the algorithm!")
    else:
        print("Nope, the dataset isn't ready yet")

The Dataset is ready, let's run the algorithm!


### Algorithm configuration

One last thing we want to do before we run our algorithm is configure the settings. In the next cell, we first print out the default settings, then make some modifications to it to use for our run of the algorithm.

In [30]:
print('Default Settings:')
pprint(algorithm['default_settings'])

settings = algorithm['default_settings']
settings['seed'] = [5,5,5]
settings['lowerThreshold'] = -1024
settings['upperThreshold'] = -750
settings['vectorRadius'] = [10, 10, 10]

print()
print('Custom Settings:')
pprint(settings)

Default Settings:
{'lowerThreshold': -1024,
 'outputContourName': 'primitive_body_contour',
 'seed': [5, 5, 5],
 'upperThreshold': -750,
 'vectorRadius': [10, 10, 10]}

Custom Settings:
{'lowerThreshold': -1024,
 'outputContourName': 'primitive_body_contour',
 'seed': [5, 5, 5],
 'upperThreshold': -750,
 'vectorRadius': [10, 10, 10]}


### Run the algorithm!

Now it's time to run our algorithm. We pass the dataset we want to run the algorithm on, the name of the algorithm and our custom settings.

Once triggered, we are given a URL to poll for the progress of the algorithm. Using this we can determine when the algorithm has finished running.

In [32]:
# Trigger the algorithm with our dataset containing the data object
data={'dataset': dataset['id'],
     'algorithm': algorithm['name'],
     'config': json.dumps(settings)}
r = requests.post(api_trigger, headers={'API_KEY': api_key}, data=data)

if r.status_code == 200:
    # Poll the URL given to determine the progress of the task
    poll_url = '{0}{1}'.format(base_url, r.json()['poll'])
    
    while(1):
        r = requests.get(poll_url, headers={'API_KEY': api_key})
        status = r.json()
        print(status)

        if status['state'] == 'SUCCESS' or status['state'] == 'FAILURE':
            break

        time.sleep(2)
else:
    print(r.json())
    
print('Algorithm Processing Complete')

{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'state': 'PENDING', 'current': 0, 'total': 1, 'status': 'Pending...'}
{'stat

### Retrieve the output

Once the algorithm finishes, we can update the dataset from the server to see what output objects we have. For those objects we are interested in, we download from the server!

In [33]:
# Fetch the latest dataset to see the output objects and download the Nifti file!
r = requests.get('{0}/{1}'.format(api_dataset, dataset['id']), headers={'API_KEY': api_key})
if r.status_code == 200:
    dataset = r.json()
    pprint(dataset)

    for d in dataset['output_data_objects']:
        if d['path'].endswith('nii.gz'):
            #print(d)
            r = requests.get('http://localhost:8000/api/dataobject/download/{0}'.format(d['id']), headers={'API_KEY': api_key})
            filename = r.headers['Content-Disposition'].split('filename=')[1]
            print('Downloading to: {0}'.format(filename))
            open(filename, 'wb').write(r.content)

{'from_dicom_location': None,
 'from_dicom_location_id': None,
 'id': 5,
 'input_data_objects': [{'children': [],
                         'dataset_id': 5,
                         'id': 6,
                         'is_fetched': True,
                         'is_input': True,
                         'is_sent': False,
                         'meta_data': None,
                         'parent': None,
                         'parent_id': None,
                         'path': '/tmp/tmp15agy_lh/case_test.nii.gz',
                         'series_instance_uid': None,
                         'status': None,
                         'timestamp': '2022-05-30T03:08:38.155',
                         'type': 'FILE'}],
 'output_data_objects': [],
 'owner_key': 'fbf59863-6d63-4e1d-9c19-b2b167f08f46',
 'timeout': '2022-05-31T03:07:57.583',
 'timestamp': '2022-05-30T03:07:57.601',
 'to_dicom_location': None,
 'to_dicom_location_id': None}


### That's it

This Notebook demonstrated the basics of running a simple segmentation algorithm on a CT image series, and downloading the resulting Nifti mask.

There is more complex stuff we can do that this. We can give the algorithm Nifti files as input, or have it automatically fetch the Dicom itself. We can also have the algorithm send generated Dicom objects (in this case RTStruct files) to a Dicom Location of our choice. Documentation and examples on how to achieve this will follow.